# Comparaison EOS diphasique

In [ ]:
from trustutils import run

run.introduction("Yannick Gorsse")
run.description("")
run.TRUST_parameters()

In [ ]:
import os
from string import Template

run.reset()
run.initBuildDirectory()
origin, build_dir = os.getcwd(), run.BUILD_DIRECTORY

fluids_template = Template("liquide_eau Fluide_generique_$lib { model $model fluid $liq } gaz_eau Fluide_generique_$lib { model $model fluid $vap } saturation_eau saturation_generique_$lib { model $model fluid $sat }")

models = {
    "EOS": {"liq": "waterliquid", "vap": "watervapor", "sat": "waterliquid", "models": ["cathare2", "refprop10"], "symbol" : "o"},
    "CoolProp": {
        "liq": "water phase liquid",
        "vap": "water phase vapor",
        "sat": "water phase liquid",
        "models": ["TTSE&HEOS", "refprop"],
        # "models": ["BICUBIC&HEOS", "TTSE&HEOS", "refprop"], # BICUBIC&HEOS ne marche pas bien!
        "symbol" : "-"
    },
}

for lib, dict_mods in models.items():
    for model in dict_mods["models"]:
        fluids = fluids_template.substitute({"lib": lib, "model": model, "liq": dict_mods["liq"], "vap": dict_mods["vap"], "sat": dict_mods["sat"]})
        run.addCaseFromTemplate("jdd.data", f"{lib}/{model.replace('&', '_')}", {"fluids": fluids})

run.runCases()
run.tablePerf()


In [ ]:
from trustutils import plot

a = plot.Graph(f"Residus")
for lib, dict_mods in models.items():
    for model in dict_mods["models"]:
        a.addResidu(f"{build_dir}/{lib}/{model.replace('&', '_')}/jdd.dt_ev", label=f"{lib} - {model}", marker=dict_mods["symbol"])

a.scale(yscale='log')

In [ ]:
from trustutils import plot

vars = {"V" : [2, 3], "P" : [0], "RHO" : [0, 1], "EINT" : [0, 1], "ALPHA" : [0], "T" : [0, 1]}

for var, compos in vars.items():
    a = plot.Graph(var, nY=len(compos))
    for lib, dict_mods in models.items():
        for model in dict_mods["models"]:
            for i, c in enumerate(compos):
                a.addPlot(i)
                a.addSegment(f"{build_dir}/{lib}/{model.replace('&', '_')}/jdd_{var}.son", label=f"{lib} - {model}", marker=dict_mods["symbol"], compo=c)
